In [1]:
import torch
from sam2.modeling_sam2 import *
from sam2.configuration_sam2 import *

/home/bingxing2/ailab/scxlab0194/.conda/envs/locr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
config = Sam2ImageEncoderConfig(embed_dim=114)

In [23]:
weight = torch.load('../../checkpoints/UparxiveV2/FlougatU1KS/flougatU_1KS_bf16_more/best/weight/epoch0018/pytorch_model.bin')

In [24]:
for k, v in weight.items():
    print(f"{k} => {v.shape}")

encoder.image_projection => torch.Size([1024, 768])
encoder.vision_tower.convs.0.proj.weight => torch.Size([128, 3, 7, 7])
encoder.vision_tower.convs.0.proj.bias => torch.Size([128])
encoder.vision_tower.convs.0.norm.weight => torch.Size([128])
encoder.vision_tower.convs.0.norm.bias => torch.Size([128])
encoder.vision_tower.convs.1.proj.weight => torch.Size([256, 128, 3, 3])
encoder.vision_tower.convs.1.proj.bias => torch.Size([256])
encoder.vision_tower.convs.1.norm.weight => torch.Size([128])
encoder.vision_tower.convs.1.norm.bias => torch.Size([128])
encoder.vision_tower.convs.2.proj.weight => torch.Size([512, 256, 3, 3])
encoder.vision_tower.convs.2.proj.bias => torch.Size([512])
encoder.vision_tower.convs.2.norm.weight => torch.Size([256])
encoder.vision_tower.convs.2.norm.bias => torch.Size([256])
encoder.vision_tower.convs.3.proj.weight => torch.Size([1024, 512, 3, 3])
encoder.vision_tower.convs.3.proj.bias => torch.Size([1024])
encoder.vision_tower.convs.3.norm.weight => torch.

In [27]:
decoder_weight = {}
for key, val in weight.items():
    if key.startswith("decoder."):
        decoder_weight[key] = val

In [36]:
torch.save(encoder_weight|decoder_weight, "../../pretrain_weights/slougat.start.pt")

In [35]:
whole_weight = encoder_weight|decoder_weight

In [33]:
whole_weight.keys()

dict_keys(['encoder.neck.convs.0.conv.bias', 'encoder.neck.convs.0.conv.weight', 'encoder.neck.convs.1.conv.bias', 'encoder.neck.convs.1.conv.weight', 'encoder.neck.convs.2.conv.bias', 'encoder.neck.convs.2.conv.weight', 'encoder.neck.convs.3.conv.bias', 'encoder.neck.convs.3.conv.weight', 'encoder.trunk.blocks.0.attn.proj.bias', 'encoder.trunk.blocks.0.attn.proj.weight', 'encoder.trunk.blocks.0.attn.qkv.bias', 'encoder.trunk.blocks.0.attn.qkv.weight', 'encoder.trunk.blocks.0.mlp.layers.0.bias', 'encoder.trunk.blocks.0.mlp.layers.0.weight', 'encoder.trunk.blocks.0.mlp.layers.1.bias', 'encoder.trunk.blocks.0.mlp.layers.1.weight', 'encoder.trunk.blocks.0.norm1.bias', 'encoder.trunk.blocks.0.norm1.weight', 'encoder.trunk.blocks.0.norm2.bias', 'encoder.trunk.blocks.0.norm2.weight', 'encoder.trunk.blocks.1.attn.proj.bias', 'encoder.trunk.blocks.1.attn.proj.weight', 'encoder.trunk.blocks.1.attn.qkv.bias', 'encoder.trunk.blocks.1.attn.qkv.weight', 'encoder.trunk.blocks.1.mlp.layers.0.bias', '

In [20]:
model = Sam2ImageEncoder(config)

AssertionError: Channel dims of trunk and neck do not match. Trunk: [1776, 888, 444, 222], neck: [896, 448, 224, 112]

In [29]:
from safetensors.torch import load_file
path = "../../pretrain_weights/sam2/model.safetensors"
weight = load_file(path)

In [34]:
encoder_weight = {}
for key, val in weight.items():
    if key.startswith("image_encoder."):
        key = "encoder.sam_image_encoder."+key[len("image_encoder."):]
        encoder_weight[key] = val

In [17]:
model.load_state_dict(encoder_weight)

RuntimeError: Error(s) in loading state_dict for Sam2ImageEncoder:
	size mismatch for neck.convs.0.conv.weight: copying a param with shape torch.Size([256, 896, 1, 1]) from checkpoint, the shape in current model is torch.Size([768, 896, 1, 1]).
	size mismatch for neck.convs.0.conv.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for neck.convs.1.conv.weight: copying a param with shape torch.Size([256, 448, 1, 1]) from checkpoint, the shape in current model is torch.Size([768, 448, 1, 1]).
	size mismatch for neck.convs.1.conv.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for neck.convs.2.conv.weight: copying a param with shape torch.Size([256, 224, 1, 1]) from checkpoint, the shape in current model is torch.Size([768, 224, 1, 1]).
	size mismatch for neck.convs.2.conv.bias: copying a param with shape torch.Size([256]) fro

In [22]:
torch.save(encoder_weight, "../../pretrain_weights/sam2_image_encoder.pt")

In [7]:
image = torch.randn(1,3,1024,768).cuda()
model = model.cuda()

In [8]:
with torch.no_grad():
    out = model(image)

In [9]:
out.keys()

dict_keys(['vision_features', 'vision_pos_enc', 'backbone_fpn'])

In [10]:
out['vision_features'].shape

torch.Size([1, 256, 64, 48])

In [43]:
out['vision_pos_enc'][2].shape

torch.Size([1, 256, 64, 48])

In [47]:
out['backbone_fpn'][2].shape

torch.Size([1, 256, 64, 48])

In [25]:

    for k, v in out.items():
        print(f"{k} => {v.shape}")
    

vision_features => torch.Size([1, 256, 64, 64])


AttributeError: 'list' object has no attribute 'shape'